<a href="https://colab.research.google.com/github/plaban1981/Agents/blob/main/Crewai_llamaindex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Install Required Dependencies

In [20]:
!pip install llama-index
!pip install llama-index-llms-groq
!pip install llama-index-core
!pip install llama-index-readers-file
!pip install llama-index-tools-wolfram-alpha
!pip install llama-index-embeddings-huggingface
!pip install 'crewai[tools]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 12.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

####Setup the LLM

In [18]:
from google.colab import userdata
from llama_index.llms.groq import Groq
groq_api_key = userdata.get('GROQ_API_KEY')
#
llm = Groq(model="llama3-70b-8192", api_key=groq_api_key)
#
response = llm.complete("Explain the importance of low latency LLMs")
print(response)
#

Low-latency Large Language Models (LLMs) are crucial in various applications where real-time or near-real-time processing is essential. Here are some reasons why low-latency LLMs are important:

1. **Interactive Systems**: In interactive systems like chatbots, virtual assistants, and conversational AI, low-latency LLMs enable rapid response times, making the interaction feel more natural and human-like. This is particularly important in applications where users expect immediate responses, such as customer support or language translation.
2. **Real-time Decision Making**: In applications like autonomous vehicles, robotics, or medical diagnosis, low-latency LLMs can process and analyze vast amounts of data in real-time, enabling swift decision-making and reaction to changing circumstances.
3. **Live Streaming and Broadcasting**: Low-latency LLMs can facilitate real-time language translation, sentiment analysis, or content moderation in live streaming and broadcasting applications, enhanc

## Setup a Simple Financial Analyst Agent

In [3]:
!wget "https://s23.q4cdn.com/407969754/files/doc_financials/2019/ar/Uber-Technologies-Inc-2019-Annual-Report.pdf" -O uber_10k.pdf

--2024-06-30 13:36:17--  https://s23.q4cdn.com/407969754/files/doc_financials/2019/ar/Uber-Technologies-Inc-2019-Annual-Report.pdf
Resolving s23.q4cdn.com (s23.q4cdn.com)... 185.172.149.128, 2a0b:4d07:201::1
Connecting to s23.q4cdn.com (s23.q4cdn.com)|185.172.149.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2829436 (2.7M) [application/pdf]
Saving to: ‘uber_10k.pdf’

uber_10k.pdf        100%[===================>]   2.70M  3.13MB/s    in 0.9s    

2024-06-30 13:36:18 (3.13 MB/s) - ‘uber_10k.pdf’ saved [2829436/2829436]



#### Read the data

In [15]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.openai import OpenAI
import os
from langchain_openai import ChatOpenAI

#
reader = SimpleDirectoryReader(input_files=["uber_10k.pdf"])
docs = reader.load_data()

In [8]:
docs[1]

Document(id_='dd161725-2512-4b03-a689-accc69dc46d4', embedding=None, metadata={'page_label': '2', 'file_name': 'uber_10k.pdf', 'file_path': 'uber_10k.pdf', 'file_type': 'application/pdf', 'file_size': 2829436, 'creation_date': '2024-06-30', 'last_modified_date': '2020-03-31'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='69  \nCountries\n10K+  \nCities\n$65B  \nGross Bookings\n111M  \nMAPCs\n7B  \nTripsA global tech \nplatform at \nmassive scale\nServing multiple multi-trillion \ndollar markets with products \nleveraging our core technology \nand infrastructure\nWe believe deeply in our bold mission. Every minute \nof every day, consumers and Drivers on our platform \ncan tap a button and get a ride or tap a button and \nget work. We revolutionized pe

In [9]:
docs[1].text

'69  \nCountries\n10K+  \nCities\n$65B  \nGross Bookings\n111M  \nMAPCs\n7B  \nTripsA global tech \nplatform at \nmassive scale\nServing multiple multi-trillion \ndollar markets with products \nleveraging our core technology \nand infrastructure\nWe believe deeply in our bold mission. Every minute \nof every day, consumers and Drivers on our platform \ncan tap a button and get a ride or tap a button and \nget work. We revolutionized personal mobility with \nridesharing, and we are leveraging our platform to \nredefine the massive meal delivery and logistics \nindustries. The foundation of our platform is our \nmassive network, leading technology, operational \nexcellence, and product expertise. Together, these \nelements power movement from point A to point B.'

In [21]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# loads BAAI/bge-small-en
# embed_model = HuggingFaceEmbedding()

# loads BAAI/bge-small-en-v1.5
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#### Create Index

In [25]:
index = VectorStoreIndex.from_documents(docs,
                                        embed_model=embed_model,
)


#### Build the Query Engine

In [26]:
query_engine = index.as_query_engine(similarity_top_k=5, llm=llm)

#### Instantiate the query engine as tool

In [27]:
from crewai_tools import LlamaIndexTool
query_tool = LlamaIndexTool.from_query_engine(
    query_engine,
    name="Uber 2019 10K Query Tool",
    description="Use this tool to lookup the 2019 Uber 10K Annual Report",
)

In [28]:
query_tool.args_schema.schema()

{'title': 'QueryToolSchema',
 'description': 'Schema for query tool.',
 'type': 'object',
 'properties': {'query': {'title': 'Query',
   'description': 'Search query for the query tool.',
   'type': 'string'}},
 'required': ['query']}

#### Create an Financial RAG Agents

In [29]:
import os
from crewai import Agent, Task, Crew, Process

In [35]:
from langchain_openai import ChatOpenAI

chat_llm = ChatOpenAI(
    openai_api_base="https://api.groq.com/openai/v1",
    openai_api_key=groq_api_key,
    model="llama3-70b-8192",
    temperature=0,
    max_tokens=1000,
)

#### Define Agents

In [36]:
# Define your agents with roles and goals
researcher = Agent(
    role="Senior Financial Analyst",
    goal="Uncover insights about different tech companies",
    backstory="""You work at an asset management firm.
  Your goal is to understand tech stocks like Uber.""",
    verbose=True,
    allow_delegation=False,
    tools=[query_tool],
    llm=chat_llm,
)
writer = Agent(
    role="Tech Content Strategist",
    goal="Craft compelling content on tech advancements",
    backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
    llm=chat_llm,
    verbose=True,
    allow_delegation=False,
)

#### Define Tasks

In [37]:
# Create tasks for your agents
task1 = Task(
    description="""Conduct a comprehensive analysis of Uber's risk factors in 2019.""",
    expected_output="Full analysis report in bullet points",
    agent=researcher,
)

task2 = Task(
    description="""Using the insights provided, develop an engaging blog
  post that highlights the headwinds that Uber faces.
  Your post should be informative yet accessible, catering to a casual audience.
  Make it sound cool, avoid complex words.""",
    expected_output="Full blog post of at least 4 paragraphs",
    agent=writer,
)


#### Instantiate your crew with a sequential process

In [38]:

crew = Crew(
    agents=[researcher, writer],
    tasks=[task1, task2],
    verbose=2,  # You can set it to 1 or 2 to different logging levels
)

#### Kickoff the Crew

In [39]:
# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)

 [2024-06-30 16:41:45][DEBUG]: == Working Agent: Senior Financial Analyst
 [2024-06-30 16:41:45][INFO]: == Starting Task: Conduct a comprehensive analysis of Uber's risk factors in 2019.


> Entering new CrewAgentExecutor chain...
I need to understand the risk factors associated with Uber in 2019.

Action: Uber 2019 10K Query Tool
Action Input: {"query": "Item 1A. Risk Factors"} 

Our business, financial condition, operating results, and prospects are exposed to various risks and uncertainties, including those related to our expansion into new markets, development of new technologies, and growth of our network of vertical take-off and landing vehicles. These risks may have a material adverse effect on our business, financial condition, and results of operations.

Thought:
I need to understand the specific risks associated with Uber's expansion into new markets.

Action: Uber 2019 10K Query Tool
Action Input: {"query": "expansion into new markets"} 

The company believes that European s